<a href="https://colab.research.google.com/github/rajy4683/EVAP2/blob/master/SRGAN_Drones_Inline2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SRGANs
Reference: https://arxiv.org/pdf/1511.06434.pdf

In [ ]:
!nvidia-smi

Sat Oct 10 06:04:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Run the comment below only when using Google Colab
!pip install torch==1.5.0+cu92 torchvision==0.6.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install torchvision-0.6.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 603.7MB 29kB/s 
     |████████████████████████████████| 6.5MB 53.1MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101


In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
import numpy as np
import datetime
import os, sys
import glob
from PIL import Image
from matplotlib import pyplot as plt

In [ ]:
from matplotlib.pyplot import imshow, imsave
%matplotlib inline
# Set random seed for reproducibility
import random
manualSeed = 0xdeadbeef
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  3735928559


In [ ]:
MODEL_NAME = 'R1GAN'
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#!cp /content/drive/My\ Drive/EVA4/Flying\ Birds.zip /content/FlyingBirds.zip
#!ls -1 /content/*zip |while read line; do unzip -q $line;done

# !cp /content/drive/My\ Drive/EVA4/Small\ QuadCopters.zip /content/SmallQuadCopters.zip
# !unzip -q /content/SmallQuadCopters.zip


#!cp /content/drive/My\ Drive/EVA4/S2GAN/allCars_wbg.tar.gz /content
#!tar -zxvf allCars_wbg.tar.gz
#!tar -zxvf allCars_wbg.tar.gz

!cp /content/drive/My\ Drive/EVA4/Flying\ Birds.zip /content/FlyingBirds.zip
!cp /content/drive/My\ Drive/EVA4/Large\ QuadCopters-20200723T085054Z-001.zip /content/LargeQuadCopters.zip
!cp /content/drive/My\ Drive/EVA4/Small\ QuadCopters.zip /content/SmallQuadCopters.zip
!cp /content/drive/My\ Drive/EVA4/Winged\ Drones-20200723T075457Z-001.zip /content/WingedDrones.zip
!cp -Rf /content/drive/My\ Drive/EVA4/SRGAN /content

!ls -1 /content/*zip |while read line; do unzip -q $line;done

Mounted at /content/drive


In [ ]:
import sys
sys.path.append("/content/drive/My Drive/EVA4/RekogNizer")
sys.path.append("/content/drive/My Drive/EVA4/")
sys.path.append("/content/SRGAN")

In [ ]:
!rm -rf /content/Winged\ Drones/*.txt
!rm -rf /content/Winged\ Drones/*.ipynb
!rm -rf /content/Winged\ Drones/*.svg

!rm -rf /content/Large\ QuadCopters/*.txt
!rm -rf /content/Large\ QuadCopters/*.ipynb
!rm -rf /content/Large\ QuadCopters/*.svg

!rm -rf /content/Small\ QuadCopters/*.txt
!rm -rf /content/Small\ QuadCopters/*.ipynb
!rm -rf /content/Small\ QuadCopters/*.svg

!rm -rf /content/Flying\ Birds/*.txt
!rm -rf /content/Flying\ Birdss/*.ipynb
!rm -rf /content/Flying\ Birds/*.svg


In [ ]:
!cp -Rf  /content/drive/My\ Drive/EVA4/SRGAN /content

In [ ]:
%load_ext autoreload
%autoreload 2
from SRGAN import data_utils
import shutil
def handle_image_errors(file_list):
    crop_size = 100
    my_transform = data_utils.train_hr_transform(crop_size)
    for file_name in file_list:
        hr_image = Image.open(file_name)
        try:        
            transformed_img = my_transform(hr_image)
            #if(transformed_img.shape[0] < 3 or transformed_img.shape[0] > 3):
            #    raise Exception
        except Exception as e:
            print(e, file_name,hr_image.size)
            shutil.move(file_name, "/content/rejected_pics/")
            continue

if not os.path.exists("/content/rejected_pics/"):
    os.makedirs("/content/rejected_pics/")

train_dir_list = ["/content/Flying Birds","/content/Large QuadCopters", 
                  "/content/Small QuadCopters", "/content/Winged Drones"]

for train_dir_name in train_dir_list:
    handle_image_errors(glob.glob(train_dir_name+"/*"))

empty range for randrange() (0,-75, -75) /content/Flying Birds/image8.png (24, 24)
empty range for randrange() (0,0, 0) /content/Flying Birds/australian-magpie-preparing-to-fly-pic.jpg (150, 99)
empty range for randrange() (0,-16, -16) /content/Large QuadCopters/this-guy-skydived-using-a-mega-drone-and-its-harr-2-12924-1494889159-0.jpg (125, 83)


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


empty range for randrange() (0,-13, -13) /content/Small QuadCopters/OIP.0tZOFuhpMDPYprQnkzB-QAHaDt.jpeg (173, 86)
empty range for randrange() (0,-9, -9) /content/Small QuadCopters/OIP - 2020-07-21T024556.887.jpg (90, 90)
empty range for randrange() (0,-12, -12) /content/Small QuadCopters/OIP (61).jpg (130, 87)
empty range for randrange() (0,-6, -6) /content/Small QuadCopters/OIP.ej5bmjJV0Ut6Fcug3rBJ6AHaEK.jpeg (166, 93)
empty range for randrange() (0,-6, -6) /content/Small QuadCopters/OIP.48HpJXdyEUHuO8-IK77YJAHaEK.jpeg (166, 93)
empty range for randrange() (0,-6, -6) /content/Small QuadCopters/OIP.Mo7TU83CQApkIuyXfQZpCwHaEK.jpeg (166, 93)
empty range for randrange() (0,-24, -24) /content/Small QuadCopters/6fec6a8e6f9be9342da45b56263598d9.jpg (75, 75)
empty range for randrange() (0,-2, -2) /content/Small QuadCopters/OIP.ztnWbehUFlxPU-kPfyzGkQHaEK.jpeg (173, 97)
empty range for randrange() (0,-7, -7) /content/Small QuadCopters/OIP - 2020-07-21T024555.791.jpg (207, 92)
empty range for ra

In [ ]:
%load_ext autoreload
%autoreload 2
from SRGAN import data_utils

In [ ]:
!python SRGAN/train_lite.py --crop_size 100 --upscale_factor 8 --num_epochs 40 --batch_size 32

Run Identifier:106e761311
# generator parameters: 881932
# discriminator parameters: 5215425
[1/1] Loss_D: 0.9181 Loss_G: 0.0760 D(x): 0.4790 D(G(z)): 0.3527:   1% 4/667 [00:03<10:55,  1.01it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
[1/1] Loss_D: 0.8938 Loss_G: 0.0728 D(x): 0.4961 D(G(z)): 0.3336:   1% 5/667 [00:03<09:51,  1.12it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
[1/1] Loss_D: 0.5126 Loss_G: 0.0400 D(x): 0.7142 D(G(z)): 0.2260:   5% 33/667 [00:19<04:36,  2.30it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA image

In [ ]:
!cp /content/statistics/* /content/drive/My\ Drive/EVA4/SRGAN_res

cp: cannot stat '/content/statistics/*': No such file or directory


In [ ]:
!ls -rlt /content/drive/My\ Drive/EVA4/SRGAN_Res/*

ls: cannot access '/content/drive/My Drive/EVA4/SRGAN_Res/*': Not a directory


In [ ]:
!cp -Rf $(ls -1rt /content/epochs/*netG* |tail -4) /content/drive/My\ Drive/EVA4/SRGAN_res

In [ ]:
!python SRGAN/train_lite.py --crop_size 100 --upscale_factor 8 --num_epochs 100 --batch_size 32

Run Identifier:90d6877c88
# generator parameters: 881932
# discriminator parameters: 5215425
[1/1] Loss_D: 0.4888 Loss_G: 0.0264 D(x): 0.7121 D(G(z)): 0.1810:  15% 98/667 [00:55<06:37,  1.43it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
[1/1] Loss_D: 0.4896 Loss_G: 0.0219 D(x): 0.7480 D(G(z)): 0.2141:  25% 169/667 [01:34<05:04,  1.63it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
[1/1] Loss_D: 0.4183 Loss_G: 0.0206 D(x): 0.7800 D(G(z)): 0.1749:  31% 209/667 [01:57<03:26,  2.22it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA i

In [ ]:
!cp -Rf $(ls -1rt /content/epochs/*netG* |tail -4) /content/drive/My\ Drive/EVA4/SRGAN_res

In [ ]:
!echo $(ls -1rt /content/epochs/*netG* |tail -2)

/content/epochs/302821e380_netG_epoch_8_1.pth /content/epochs/302821e380_netG_epoch_8_2.pth


In [ ]:
!rm -rf /content/drive/My\ Drive/EVA4/SRGAN_Res/302821e380_netG_epoch_8_2.pth

In [ ]:
!python SRGAN/test_image.py --upscale_factor 8 --image_name "/content/Winged Drones/022.jpg" --model_name "94625922b6_netG_epoch_8_5.pth"

In [ ]:
!rm -rf /content/Flying Birds/Lc58dvGVZKA.jpg /content/Large QuadCopters/jordan-cormack-s6L_CmBiDQw-unsplash.jpg /content/Flying Birds/2bHXm5Ae_rY.jpg /content/Flying Birds/jzvA1hIRU3o.jpg /content/Flying Birds/000002.jpg /content/Flying Birds/white-and-black-bird-flying-4212935.jpg /content/Winged Drones/Image_930.jpg

In [ ]:
!python SRGAN/train_lite.py --crop_size 100 --upscale_factor 8 --num_epochs 40 --batch_size 32

Run Identifier:1706b1f481
# generator parameters: 881932
# discriminator parameters: 5215425
[1/1] Loss_D: 0.7191 Loss_G: 0.0497 D(x): 0.6098 D(G(z)): 0.2891:   3% 19/667 [00:11<04:24,  2.45it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
[1/1] Loss_D: 0.7043 Loss_G: 0.0452 D(x): 0.6159 D(G(z)): 0.2745:   4% 24/667 [00:14<04:47,  2.24it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
[1/1] Loss_D: 0.6786 Loss_G: 0.0402 D(x): 0.6278 D(G(z)): 0.2606:   5% 32/667 [00:16<03:26,  3.08it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA ima

In [ ]:
!python SRGAN/train_lite.py --crop_size 100 --upscale_factor 8 --num_epochs 60 --batch_size 32

In [ ]:
!python SRGAN/test_image.py --upscale_factor 4 --image_name "/content/Large QuadCopters/OIP_i2.jpg" --model_name 2deb120d9c_netG_epoch_4_9.pth

SRGAN/test_image.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  image = Variable(ToTensor()(image), volatile=True).unsqueeze(0)
cost0.09877999999999965s


In [ ]:
!python SRGAN/train.py --crop_size 100 --upscale_factor 4 --num_epochs 100 --train_dir "/content/Small_QuadCopters" --val_dir "/content/Small_QuadCopters_val"

# generator parameters: 734219
# discriminator parameters: 5215425
[1/100] Loss_D: 0.4957 Loss_G: 0.0242 D(x): 0.7298 D(G(z)): 0.2255: 100% 51/51 [00:30<00:00,  1.67it/s]
[converting LR images to SR images] PSNR: 19.8500 dB SSIM: 0.7115: 100% 399/399 [00:34<00:00, 11.46it/s]
[saving training results]: 100% 75/75 [00:04<00:00, 15.11it/s]
[2/100] Loss_D: 0.6296 Loss_G: 0.0137 D(x): 0.8408 D(G(z)): 0.4704: 100% 51/51 [00:30<00:00,  1.65it/s]
[converting LR images to SR images] PSNR: 18.1930 dB SSIM: 0.7154: 100% 399/399 [00:29<00:00, 13.41it/s]
[saving training results]: 100% 75/75 [00:04<00:00, 15.19it/s]
[3/100] Loss_D: 0.9869 Loss_G: 0.0117 D(x): 0.7821 D(G(z)): 0.7689: 100% 51/51 [00:30<00:00,  1.69it/s]
[converting LR images to SR images] PSNR: 19.0274 dB SSIM: 0.7459: 100% 399/399 [00:29<00:00, 13.72it/s]
[saving training results]: 100% 75/75 [00:04<00:00, 15.02it/s]
[4/100] Loss_D: 0.9953 Loss_G: 0.0096 D(x): 0.9085 D(G(z)): 0.9038: 100% 51/51 [00:30<00:00,  1.68it/s]
[converting L

In [ ]:
!python SRGAN/train.py --crop_size 100 --upscale_factor 4 --num_epochs 200 --train_dir "/content/Small_QuadCopters" --val_dir "/content/Small_QuadCopters_val"

# generator parameters: 734219
# discriminator parameters: 5215425
[1/200] Loss_D: 0.6561 Loss_G: 0.0266 D(x): 0.6690 D(G(z)): 0.3251: 100% 51/51 [00:30<00:00,  1.68it/s]
[converting LR images to SR images] PSNR: 16.7878 dB SSIM: 0.6734: 100% 399/399 [00:35<00:00, 11.39it/s]
[saving training results]: 100% 75/75 [00:05<00:00, 14.82it/s]
[2/200] Loss_D: 0.9964 Loss_G: 0.0152 D(x): 0.9863 D(G(z)): 0.9827: 100% 51/51 [00:30<00:00,  1.68it/s]
[converting LR images to SR images] PSNR: 21.0970 dB SSIM: 0.7430: 100% 399/399 [00:29<00:00, 13.36it/s]
[saving training results]: 100% 75/75 [00:04<00:00, 15.30it/s]
[3/200] Loss_D: 1.0008 Loss_G: 0.0116 D(x): 0.9944 D(G(z)): 0.9952: 100% 51/51 [00:30<00:00,  1.69it/s]
[converting LR images to SR images] PSNR: 22.2335 dB SSIM: 0.7799: 100% 399/399 [00:28<00:00, 13.79it/s]
[saving training results]: 100% 75/75 [00:04<00:00, 15.33it/s]
[4/200] Loss_D: 1.0000 Loss_G: 0.0096 D(x): 0.9973 D(G(z)): 0.9974: 100% 51/51 [00:30<00:00,  1.66it/s]
[converting L

In [ ]:
!ls -rlt /content/training_results/SRF_4

In [ ]:
sample_pic = os.path.join("/content/training_results/SRF_4","epoch_50_index_2.png")
Image.open(sample_pic)

In [ ]:
import torch.optim as optim
import torch.utils.data
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm

from SRGAN.data_utils import TrainDatasetFromFolder, ValDatasetFromFolder

CROP_SIZE = 100
UPSCALE_FACTOR = 2
train_set = TrainDatasetFromFolder("/content/Small_QuadCopters", crop_size=CROP_SIZE, upscale_factor=UPSCALE_FACTOR)
val_set = ValDatasetFromFolder("/content/Small_QuadCopters_val", upscale_factor=UPSCALE_FACTOR)
train_loader = DataLoader(dataset=train_set, num_workers=4, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_set, num_workers=4, batch_size=1, shuffle=False)

In [ ]:
train_bar = tqdm(train_loader)

for data, target in train_bar:
    #print(data.shape, target.shape)
    continue

In [ ]:
Image.open("/content/training_results/SRF_4/epoch_63_index_6.png")

In [ ]:
!cp -R /content/SRGAN /content/drive/My\ Drive/EVA4/

In [ ]:
import sys
sys.path.append("/content/drive/My Drive/EVA4/RekogNizer")
sys.path.append("/content/drive/My Drive/EVA4/")

In [ ]:
%load_ext autoreload
%autoreload 2
from RekogNizer import ganmodels
from RekogNizer import ganutils
from RekogNizer import vaemodelsmse


G_losses = []
D_losses = []
img_list = []
#r1gan = ganmodels.R1GAN(img_size=64,d_in_channel=IMAGE_DIM[-1],g_out_channel=IMAGE_DIM[-1],g_n_noise=256 ,model_prefix="r1gan_adam64")


In [ ]:
!mv Flying\ Birds FlyingBirds

In [ ]:
!mkdir FlyingBirdsVal

In [ ]:
!mv $(ls -1 /content/FlyingBirds/* |tail -1000) /content/FlyingBirdsVal

In [ ]:
ls -1 /content/FlyingBirdsVal/* |wc -l

1868


In [ ]:
!cp -Rf /content/drive/My\ Drive/EVA4/SRGAN_res/ .

In [ ]:
import argparse
import time
import os
import torch
from PIL import Image
from torch.autograd import Variable
from torchvision.transforms import ToTensor, ToPILImage

from model import Generator

UPSCALE_FACTOR = 4
test_mode='GPU'
TEST_MODE = True if test_mode == 'GPU' else False
#IMAGE_NAME = opt.image_name
#MODEL_NAME = opt.model_name

model = Generator(UPSCALE_FACTOR).eval()

In [ ]:
model.cuda()
model.load_state_dict(torch.load("/content/SRGAN_res/1ca3e2a23a_netG_epoch_4_25.pth"))

<All keys matched successfully>

In [ ]:
traced_model = torch.jit.trace(model.to("cpu"), torch.randn(1,3,512,512))
#torch.jit.save(traced_model, '/content/resnet_custom_facedetect_2.pt')

In [ ]:
!cp /content/drive/My\ Drive/EVA4/Winged\ Drones-20200723T075457Z-001.zip /content/WingedDrones.zip


In [ ]:
!unzip /content/WingedDrones.zip

In [ ]:
IMAGE_NAME="/content/Winged Drones/3l-image-22.jpg"
image = Image.open(IMAGE_NAME)
image = Variable(ToTensor()(image), volatile=True).unsqueeze(0)
if TEST_MODE:
    image = image.cuda()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
out = traced_model(image.to(torch.device("cpu")))
#elapsed = (time.clock() - start)
#print('cost' + str(elapsed) + 's')
out_img = ToPILImage()(out[0].data.cpu())

In [ ]:
def transform_image( image_bytes ) :
    try:
        #transformations = transforms.Compose([
        #                #transforms.Resize(224) ,
        #                #transforms.CenterCrop(224),
        #                np.float32,
        #                transforms.ToTensor(),
        #                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        #transformations = transforms.Compose([transforms.Resize((128,128)),
        #    transforms.CenterCrop(128),
        #    transforms.ToTensor(),
        #    transforms.Normalize(mean=(0.5, 0.5, 0.5),std=(0.5, 0.5, 0.5))
        #])
        #img = np.array(Image.open().convert('RGB'))
        transformations = transforms.Compose([transforms.Resize((128,128)),
            transforms.CenterCrop(128),
            transforms.ToTensor(),
        ])
        # image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        image = Image.open(image_bytes).convert("RGB")

        print("Transformations successful!")
        return transformations(image).unsqueeze(0)
    except Exception as e:
        print("Exception during transform_image", repr(e))
        raise(e)

Transformations successful!


In [ ]:
input_img.shape

torch.Size([1, 3, 128, 128])

In [ ]:
#new_noise = torch.rand((8,256))#.reshape((1,256,))
import io,base64
with torch.no_grad():
    input_img = transform_image(IMAGE_NAME)

    output_images = model(input_img)
    print("Model interpretation completed")
    mynew_arr = np.transpose(output_images[0].clone().clamp(0, 255).numpy(),(1,2,0))
    mynew_arr = np.asarray(mynew_arr*255,dtype=np.uint8)
    img = Image.fromarray(mynew_arr,mode="RGB")
    buffer = io.BytesIO()
    img.save(buffer,format="JPEG")
    myimage = base64.b64encode(buffer.getvalue())

Transformations successful!
Model interpretation completed


In [ ]:
myimage.decode()

'...'

In [ ]:
out[0].data.cpu().clamp(0,255).numpy()

In [ ]:
image_name_out = os.path.basename(IMAGE_NAME)
out_img.save('out_srf_' + str(UPSCALE_FACTOR) + '_' + image_name_out)

In [ ]:
torch.jit.save(traced_model, '/content/s8superres.pt')

In [ ]:
from RekogNizer import awsutils
saved_model = "/content/s8superres.pt"
awsutils.upload_model_to_s3('rekog-eva4s1',saved_model,os.path.basename(saved_model))

In [ ]:
!pip install boto3

     |████████████████████████████████| 133kB 8.8MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 6.7MB 14.4MB/s 
